In [1]:
import pandas as pd
import random
import datetime
from tqdm import tqdm
import requests as r
from bson import ObjectId

# 0. Entendiendo el problema

Variables de entrada:
- TPU
- Cantidad requerida
- Tiempo de produccion requerido

Salida:
- Personal requerido


Por lo que se tiene una relacion lineal multivariada:

`y = B0 + B1([Cantidad material]) + B2([Tiempo de fabricacion del material]) + E`

- B0: Cantidad base de personal
- B1: Un producto de el factor de incremento de personal por incremento de cantidad de material y el tpu
- B2: Tiempo requerido para fabricar

In [2]:
_mat = [
    'Discos de freno',
    'Pastillas de freno',
    'Calipers de freno',
    'Tambores de freno',
    'Zapatas de freno',
    'Cilindro maestro de freno',
    'Líneas de freno',
    'Sensor de freno ABS',
    'Servo o booster de freno',
    'Mangueras de freno',
    'Válvula de proporción de freno',
    'Depósito de líquido de freno',
    'Freno de mano (o freno de estacionamiento)',
    'Freno de tambor',
    'Discos ventilados',
    'Sensor de desgaste de pastillas',
    'Rotores de freno',
    'Pinza de freno',
    'Conjunto de frenado trasero',
    'Mordaza de freno',
    'Kit de resorte de tambor de freno',
    'Líquido de frenos',
    'Pistones de caliper',
    'Válvula de control de presión de freno',
    'Anillo reluctor de ABS',
    'Conector de freno de estacionamiento',
    'Pedal de freno',
    'Módulo de control de freno',
    'Tornillo de purga de freno',
    'Anillo de retención de tambor de freno',
]

In [3]:
productos = {
    f"SKU{i}": {
        '_id':ObjectId(),
        'descripcion':_mat[i],
        'tpu': random.randint(30,50),
        'personal_factor_cantidad':random.randint(2,10) / 100000,
        'personal_factor_tiempo':random.randint(1000,5000) / 100
    }
    
    for i in range(30)
}
cantidad_min, cantidad_max = 500, 5000
tiempo_min, tiempo_max = 6, 24
personal_base = 5



def generar_ot(dias_atras = 365, dias_adelante = 0, pers_req = True):
    
    producto = random.choice(list(productos.keys()))
    
    p = productos[producto]
    tpu = p['tpu']
    personal_factor_cantidad = p['personal_factor_cantidad']
    personal_factor_tiempo = p['personal_factor_tiempo']
    
    
    cantidad = random.randint(cantidad_min, cantidad_max)
    tiempo = random.uniform(tiempo_min, tiempo_max)

    personal_requerido = int(personal_base + personal_factor_cantidad * tpu * cantidad \
                             + personal_factor_tiempo / tiempo \
                             + random.uniform(-1, 1))
    
    # print(f"y = {personal_base} + {personal_factor_cantidad}*{tpu}({cantidad}) + {personal_factor_tiempo}/{tiempo:,.2f} + {random.randint(-2, 2)} = {personal_requerido:,}")

    personal_requerido = max(1, personal_requerido)

    inicio = datetime.datetime.now() + datetime.timedelta(days=random.randint(-1 * dias_atras, dias_adelante), hours=random.randint(-8,8), minutes=random.randint(-60,60))
    final = inicio + datetime.timedelta(hours=tiempo)

    ot = {
        "producto": productos[producto]['_id'],
        "cantidad": cantidad,
        "inicio": inicio.strftime("%Y-%m-%d %H:%M"),
        "final": final.strftime("%Y-%m-%d %H:%M"),
        '_pseudo': producto,
    }

    
    return {**ot, "personalAsignado": personal_requerido} if pers_req else ot

def generar_datos(n=1000, dias_atras = 365, dias_adelante = 0, pers_req=True):
    ots = [generar_ot(dias_atras=dias_atras, dias_adelante=dias_adelante, pers_req=pers_req) for _ in tqdm(range(n))]
    return ots


data = pd.DataFrame(generar_datos(250_000))

100%|██████████| 250000/250000 [00:03<00:00, 79987.47it/s]


In [4]:
data['personalAsignado'].min(),data['personalAsignado'].max()

(6, 34)

# Evaluando los resultados de los datos creados

In [5]:
data['tiempo'] = (pd.to_datetime(data['final']) - pd.to_datetime(data['inicio'])).dt.total_seconds() / 3600

In [6]:
SKU = "SKU15"

In [7]:
import pandas as pd
import plotly.graph_objects as go

# Supongamos que tienes este DataFrame:
df = data[data['_pseudo']==SKU]

# Crear el gráfico 3D interactivo
fig = go.Figure(data=go.Scatter3d(
    x=df['cantidad'],
    y=df['tiempo'],
    z=df['personalAsignado'],
    mode='markers',
    marker=dict(
        size=1,
        color=df['personalAsignado'],  # Color según personal asignado
        colorscale='rdylbu',
        opacity=0.8
    )
))

# Personalizar el gráfico
fig.update_layout(
    scene = dict(
        xaxis_title='Cantidad',
        yaxis_title='Tiempo',
        zaxis_title='Personal Asignado'
    ),
    title=f'TPU = {productos[SKU]["tpu"]}'
)

# Mostrar el gráfico
fig.show()

# 1. Generando datos para la populacion de la base de datos

Para las acciones de populacion de bases de datos se mantendrá un avg de OT por dia de 5, para no sobrepoblar la bd.

Se generan datos para 5 años hacia atras.

In [8]:
materiales = [{'_id':p['_id'], 'tpu':p['tpu'], 'descripcion':p['descripcion']} for k,p in productos.items()]

In [9]:
colabs = [{'_id':ObjectId(),'nombre':f"{p['name']['first']} {p['name']['last']}",'foto':p['picture']['large']} for p in r.get('https://randomuser.me/api/?results=50').json()['results']]

In [10]:
ot = generar_datos(n = (5 * 365 + 60) * 5, dias_atras=365 * 5, dias_adelante=60, pers_req=True)

100%|██████████| 9425/9425 [00:00<00:00, 77401.74it/s]


Dado que la funcion unicamente devuelve el entero de la cantidad de personas asignadas, hay que convertirlo en una lista aleatoria de colaboradores en ot

In [11]:
for item in ot:
    status = None
    if pd.to_datetime(item['inicio']) <= pd.to_datetime('now'):
        if pd.to_datetime(item['final']) <= pd.to_datetime('now'):
            status = 2
        else:
            status = 1
    else:
        status = 0

    item.pop('_pseudo')

    if status > 0:
        num_ids = int(item['personalAsignado'])
        ids_unique = random.sample([d['_id'] for d in colabs], k=num_ids)
        item['personalAsignado'] = ids_unique

    else:
        item['personalAsignado'] = None


In [12]:
ot = pd.DataFrame(ot)

# 2. Entrenamiento de modelo

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [14]:
data['tpu'] = data['_pseudo'].apply(lambda x: productos.get(x).get('tpu'))

In [15]:
X = data[['tiempo','tpu','cantidad']]
y = data['personalAsignado']

### 2.1 Train test split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

### 2.2.1 Modelo lineal

In [17]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [18]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5

print(f'Coeficiente R^2: {r2}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

Coeficiente R^2: 0.6142086056938141
MAE: 2.4465176704527702
MSE: 9.272633261343568
RMSE: 3.045099877071944


### 2.2.2 Modelo Random Forest

In [19]:
model2 = RandomForestRegressor()

model2.fit(X_train, y_train)

RandomForestRegressor()

In [20]:
y_pred_rf = model2.predict(X_test)


r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)
mse = mean_squared_error(y_test, y_pred_rf)
rmse = mse**0.5

print(f'Coeficiente R^2: {r2}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

Coeficiente R^2: 0.8382331698441665
MAE: 1.1461219460317462
MSE: 3.888123250086312
RMSE: 1.9718324599433674


In [21]:
import joblib

joblib.dump(model2, "api\\services\\PA_Regressor\\random_forest.pkl")

['api\\services\\PA_Regressor\\random_forest.pkl']

### 2.2.3 Uso del modelo generado para colocar la informacion del forecast en los registros que van a ser introducidos en la BD

In [22]:
ot['tiempo'] = (pd.to_datetime(ot['final'])-pd.to_datetime(ot['inicio'])).dt.total_seconds() / 3600
ot['tpu'] = ot['producto'].apply(lambda x: list(filter(lambda a: a['_id']==x, materiales))[0]['tpu'])

In [23]:
resultados_forecast = model2.predict(ot[['tiempo','tpu','cantidad']])

ot['personalForecast'] = resultados_forecast
ot['personalForecast'] = ot['personalForecast'].astype(int)

In [24]:
ot.drop(columns=['tiempo','tpu'],inplace=True)

In [25]:
ot = ot.to_dict(orient='records')

# 3. Inserción de datos en MongoDB

In [1]:
from api.db import db

In [27]:
if input('Insertar en bd? y/n') == 'y':
    db.ordenes.insert_many(ot)
    db.materiales.insert_many(materiales)
    db.plantilla.insert_many(colabs)

In [13]:
from api.services.MongoDB_Pipes.MongoDB_Pipes import OT_With_Full_Data


s = list(db.ordenes.aggregate(OT_With_Full_Data([{"inicio":{"$lte":"2024-10-27 19:41"}}, {"final":{"$gte":"2024-10-20 19:41"}}])))

[{'$addFields': {'productoOID': {'$toObjectId': '$producto'}}}, {'$lookup': {'from': 'materiales', 'localField': 'productoOID', 'foreignField': '_id', 'as': 'productoData'}}, {'$unwind': {'path': '$productoData'}}, {'$unwind': {'path': '$personalAsignado', 'preserveNullAndEmptyArrays': True}}, {'$lookup': {'from': 'plantilla', 'localField': 'personalAsignado', 'foreignField': '_id', 'as': 'personalAsignado'}}, {'$unwind': {'path': '$personalAsignado', 'preserveNullAndEmptyArrays': True}}, {'$addFields': {'personalAsignado._id': {'$toString': '$personalAsignado._id'}}}, {'$addFields': {'productoNombre': '$productoData.descripcion'}}, {'$group': {'_id': '$_id', 'producto': {'$first': '$producto'}, 'descripcion': {'$first': '$productoNombre'}, 'cantidad': {'$first': '$cantidad'}, 'inicio': {'$first': '$inicio'}, 'final': {'$first': '$final'}, 'personalAsignado': {'$push': '$personalAsignado'}, 'personalForecast': {'$first': '$personalForecast'}}}, {'$project': {'_id': 1, 'producto': 1, 'd

AttributeError: 'CommandCursor' object has no attribute 'limit'

In [8]:
import pandas as pd

In [9]:
s = pd.DataFrame(s)

In [10]:
s['p'] = s['personalAsignado'].apply(len)

In [11]:
s['diff'] = s['p']-s['personalForecast']